# Location Cleaning Framework

Braden Riggs A15089134 DSC198

Additional geodata provided by Geonames.org

In [ ]:
#Import packages for cleaning
#!pip install pycountry
#!pip install geopy
import pandas as pd
import numpy as np
import pycountry
from geopy.geocoders import Nominatim
pd.options.display.max_columns = None 

Enter file for standardization:

In [ ]:
std_file = "data/CNCB"

Enter location column names(s):

In [ ]:
loc1 = "Location"
loc2 = "none"
loc3 = "none"
loc4 = "none"

Run Below Code:

In [ ]:
metadata_url = "https://bigd.big.ac.cn/ncov/genome/export/meta"
df = pd.read_excel(metadata_url, dtype='str')
df.fillna('', inplace=True)

In [ ]:
#LOAD DATA AS DF
#df = pd.read_csv(std_file)
#df = df[:100] #For testing
df.head()

## Step #1 Geocode initial location

In [ ]:
# Row filtering and organizing data
def order_geocode(datafrm, loc1, loc2="none", loc3="none", loc4="none"):
    if isinstance(datafrm, pd.DataFrame):
        if loc2 != "none" and loc2 not in list(datafrm.columns):
            print("Error loc2 is an invalid column name!")
        if loc3 != "none" and loc3 not in list(datafrm.columns):
            print("Error loc3 is an invalid column name!")
        if loc4 != "none" and loc4 not in list(datafrm.columns):
            print("Error loc4 is an invalid column name!")
        
        
        if loc1 in list(datafrm.columns):
            datafrm["location_series"] = datafrm[loc1]
            if loc2 != "none":
                datafrm["location_series"] = datafrm.location_series.map(str) + ", " + datafrm["loc2"].map(str)
            if loc3 != "none":
                datafrm["location_series"] = datafrm.location_series.map(str) + ", " + datafrm["loc3"].map(str)
            if loc4 != "none":
                datafrm["location_series"] = datafrm.location_series.map(str) + ", " + datafrm["loc4"].map(str)
                
            grouped_loc = datafrm.groupby("location_series").count()
            
            holder1 = []
            c = 0
            for i in grouped_loc.index:
                temp = str(do_geocode(text_cleaner(i)))
                print(str(c) + ", ", end = '')
                temp = temp.split(",")
                holder2 = []
                for k in temp:
                    holder2.insert(0,text_cleaner(k))
                holder1.append(holder2)
                c = c + 1
            print(grouped_loc.shape)
            locs = pd.DataFrame(holder1, columns = ["Country","Zip_Code","Admin1","Admin2","Admin3","Admin4","Admin5","Admin6","Admin7","Admin8", "Admin9","Admin10"])
            print(locs.shape)
            locs["location_series"] = grouped_loc.index
                
            return locs
                
        else:
            print("Loc1 must be a valid column name!")
    else:
        print("Datafrm must be a pandas dataframe type object!")
    

In [ ]:
geolocator = Nominatim(user_agent="covid19_geocoding")
from geopy.exc import GeocoderTimedOut

#Geocodes
def do_geocode(address):
    try:
        return geolocator.geocode(address,addressdetails=True, geometry='wkt', extratags=True, timeout = 5, language = "en-US")
    except GeocoderTimedOut:
        return do_geocode(address)

In [ ]:
def text_cleaner(text):
    text = str(text).lower()
    text = text.strip()
    text = text.replace("-"," ")
    text = text.replace(","," ")
    text = text.replace("!","")
    text = text.replace("ä","a")
    text = text.replace("ó","o")
    text = text.replace("è","e")
    text = text.replace(":",",")
    return text

In [ ]:
temp = order_geocode(df,loc1)
df = pd.merge(df, temp, how='inner',left_on="location_series", right_on="location_series")
df.head(3)

In [ ]:
df.to_csv(std_file + "_GeoCoded", header=True, index=True)

## Step #2 Match on Geonames Data for Country

In [ ]:
#Load back in saved data
df = pd.read_csv(std_file + "_GeoCoded",  index_col=0)

In [ ]:
countries = pd.read_csv("data/countryInfo.txt", sep='\t', skiprows=(np.arange(49))) #Skip metadata
countries = countries[["Country","#ISO","ISO3","ISO-Numeric","fips","geonameid","neighbours"]] #Filter for relevent data
countries = countries.rename(columns = {"neighbours" : "Neighbouring_Country"})
countries["Country"] = countries["Country"].str.lower()
countries.head(5)

In [ ]:
#None type entry
noner = pd.Series(["none","None","None","None","None","None","None"], index = ["Country","#ISO","ISO3","ISO-Numeric","fips","geonameid","Neighbouring_Country"])


countries["Country"] = countries["Country"].replace({"czechia":"czech republic", "united states":"united states of america","netherlands":"the netherlands"})
df["Country"] = df["Country"].replace({"united states":"united states of america","czechia":"czech republic","netherlands":"the netherlands"})
countries = countries.append(noner, ignore_index = True)


hold= []
for i in df["Country"].unique():
    temp = 0
    for j in countries["Country"].unique():
        if i == j:
            hold.append(i)
            temp = 1
            break
    if temp != 1: #USA is called united states and Czech Republic is called Czechia
        print(i)

In [ ]:
print(df.shape)
df_country = pd.merge(df, countries, how='inner',left_on="Country", right_on="Country")
df_country
print(df_country.shape) #Evaluating loss

In [ ]:
df_country

In [ ]:
df_country.to_csv(std_file + "_GeoNamed", header=True, index=True)

## Step #3 Match on Geonames Data for Admin 1

In [ ]:
geodata_df = pd.read_csv(std_file + "_GeoCoded")
geodata_df.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
geodata_df